In [1]:
#!/usr/bin/env python3
from std_srvs.srv import Empty, Trigger, TriggerRequest
import smach
from move_base_msgs.msg import MoveBaseAction, MoveBaseGoal
from geometry_msgs.msg import PoseStamped, Point , Quaternion
from actionlib_msgs.msg import GoalStatus
import moveit_commander
import moveit_msgs.msg
import tf2_ros
from tf2_sensor_msgs.tf2_sensor_msgs import do_transform_cloud
#from object_classification.srv import *
from sensor_msgs.msg import Image as ImageMsg
#from cv_bridge import CvBridge, CvBridgeError

from utils_notebooks import *
from utils_takeshi import *
def pose_2_np(wp_p):
    
    #Takes a pose message and returns array
   
    return np.asarray((wp_p.pose.position.x,wp_p.pose.position.y,wp_p.pose.position.z)),np.asarray((wp_p.pose.orientation.w,wp_p.pose.orientation.x,wp_p.pose.orientation.y, wp_p.pose.orientation.z)) 
def np_2_pose(position,orientation):
    #Takes a pose np array and returns pose message
    wb_p= geometry_msgs.msg.PoseStamped()
    
    wb_p.pose.position.x= position[0]
    wb_p.pose.position.y= position[1]
    wb_p.pose.position.z= position[2]
    wb_p.pose.orientation.w= orientation[0]
    wb_p.pose.orientation.x= orientation[1]
    wb_p.pose.orientation.y= orientation[2]
    wb_p.pose.orientation.z= orientation[3]
    return wb_p

def open_gripper():
    gcv=gripper.get_current_joint_values()
    gcv[2]= 0.8
    gripper.set_joint_value_target(gcv)
    gripper.go(gcv)
def close_gripper():
    gcv=gripper.get_current_joint_values()
    gcv[2]= 0.0
    gripper.set_joint_value_target(gcv)
    gripper.go(gcv)
def move_to_joint_value(goal):
    joint_goal = arm.get_current_joint_values()
    #print(joint_goal)
    for i in range (5):
        #use None when you don't want to change the joint value
        if goal[i] is not None: 
            joint_goal[i] = goal[i]
    try:
        arm.go(joint_goal)
        arm.stop()
        arm.clear_pose_targets()
        return True
    except:
        return False
    
def move_to_pose_value(CG=0,posX=None,posY=None,posZ=None,w=None):
#pID=arm.set_planner_id("RRTConnectkConfigDefault")
    if CG==0:
        group=arm
    elif CG==1:
        group=whole_body
    group.allow_replanning(True)
    #set start state
    #arm.set_start_state_to_current_state()
    goal_pose = group.get_current_pose()
    #set goal state
    if posX is not None:
        goal_pose.pose.position.x = posX
    if posY is not None:
        goal_pose.pose.position.y = posY
    if posZ is not None:
        goal_pose.pose.position.z = posZ
    if w is not None:
        goal_pose.pose.orientation.w = w
    
    #print(goal_pose)
    try:
        group.set_joint_value_target(goal_pose,True)
        group.go()
        group.stop()
        group.clear_pose_targets()
        return True
    except:
        return False
def shift_position(eje,shift):
#     shift=+0.03
#     eje=2
    axis = ['x','y','z','r','p','y']
    index = axis.index(eje)
    arm.shift_pose_target(index,shift)
    succ = arm.go()
    while not(succ):
        if shift != 0:
            if shift > 0:
                shift -= 0.01
            else:
                shift += 0.01
        else:
            break
        arm.shift_pose_target(index,shift)
        succ = arm.go()
#     print(shift)
    arm.stop()
    return succ
    
def move_abs(vx,vy,vw, time=0.05):
    start_time = get_current_time_sec() 
    while get_current_time_sec() - start_time < time: 
        twist = Twist()
        twist.linear.x = vx
        twist.linear.y = vy
        twist.angular.z = vw / 180.0 * np.pi  
        base_vel_pub.publish(twist)
def get_current_time_sec():

    return rospy.Time.now().to_sec()
    

def correct_points(low=.27,high=1000):

    #Corrects point clouds "perspective" i.e. Reference frame head is changed to reference frame map
    data = rospy.wait_for_message('/hsrb/head_rgbd_sensor/depth_registered/rectified_points', PointCloud2)
    np_data=ros_numpy.numpify(data)
    trans,rot=listener.lookupTransform('/map', '/head_rgbd_sensor_gazebo_frame', rospy.Time(0)) 
    
    eu=np.asarray(tf.transformations.euler_from_quaternion(rot))
    t=TransformStamped()
    rot=tf.transformations.quaternion_from_euler(-eu[1],0,0)
    t.header.stamp = data.header.stamp
    
    t.transform.rotation.x = rot[0]
    t.transform.rotation.y = rot[1]
    t.transform.rotation.z = rot[2]
    t.transform.rotation.w = rot[3]

    cloud_out = do_transform_cloud(data, t)
    np_corrected=ros_numpy.numpify(cloud_out)
    corrected=np_corrected.reshape(np_data.shape)

    img= np.copy(corrected['y'])

    img[np.isnan(img)]=2
    img3 = np.where((img>low)&(img< 0.99*(trans[2])),img,255)
    return img3

def plane_seg_square_imgs(lower=500 ,higher=50000,reg_ly= 30,reg_hy=600,plt_images=True):

    #Segment  Plane using corrected point cloud
    #Lower, higher = min, max area of the box
    #reg_ly= 30,reg_hy=600    Region (low y  region high y ) Only centroids within region are accepted
    
    image= rgbd.get_h_image()
    iimmg= rgbd.get_image()
    points_data= rgbd.get_points()
    img=np.copy(image)
    img3= correct_points()


    _,contours, hierarchy = cv2.findContours(img3.astype('uint8'),cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    i=0
    cents=[]
    points=[]
    images=[]
    for i, contour in enumerate(contours):

        area = cv2.contourArea(contour)

        if area > lower and area < higher :
            M = cv2.moments(contour)
            # calculate x,y coordinate of center
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])


            boundRect = cv2.boundingRect(contour)
            #just for drawing rect, dont waste too much time on this
            image_aux= iimmg[boundRect[1]:boundRect[1]+max(boundRect[2],boundRect[3]),boundRect[0]:boundRect[0]+max(boundRect[2],boundRect[3])]
            images.append(image_aux)
            img=cv2.rectangle(img,(boundRect[0], boundRect[1]),(boundRect[0]+boundRect[2], boundRect[1]+boundRect[3]), (0,0,0), 2)
            # calculate moments for each contour
            if (cY > reg_ly and cY < reg_hy  ):

                cv2.circle(img, (cX, cY), 5, (255, 255, 255), -1)
                cv2.putText(img, "centroid_"+str(i)+"_"+str(cX)+','+str(cY)    ,    (cX - 25, cY - 25)   ,cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 0), 2)
                print ('cX,cY',cX,cY)
                xyz=[]


                for jy in range (boundRect[0], boundRect[0]+boundRect[2]):
                    for ix in range(boundRect[1], boundRect[1]+boundRect[3]):
                        aux=(np.asarray((points_data['x'][ix,jy],points_data['y'][ix,jy],points_data['z'][ix,jy])))
                        if np.isnan(aux[0]) or np.isnan(aux[1]) or np.isnan(aux[2]):
                            'reject point'
                        else:
                            xyz.append(aux)

                xyz=np.asarray(xyz)
                cent=xyz.mean(axis=0)
                cents.append(cent)
                print (cent)
                points.append(xyz)
            else:
                print ('cent out of region... rejected')
    sub_plt=0
    if plt_images:
        for image in images:

            sub_plt+=1
            ax = plt.subplot(5, 5, sub_plt )

            plt.imshow(image)
            plt.axis("off")

    cents=np.asarray(cents)
    ### returns centroids found and a group of 3d coordinates that conform the centroid
    return(cents,np.asarray(points), images)
def seg_square_imgs(lower=2000,higher=50000,reg_ly=0,reg_hy=1000,reg_lx=0,reg_hx=1000,plt_images=False): 

    #Using kmeans for image segmentation find
    #Lower, higher = min, max area of the box
    #reg_ly= 30,reg_hy=600,reg_lx=0,reg_hx=1000,    Region (low  x,y  region high x,y ) Only centroids within region are accepted
    image= rgbd.get_h_image()
    iimmg= rgbd.get_image()
    points_data= rgbd.get_points()
    values=image.reshape((-1,3))
    values= np.float32(values)
    criteria= (  cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER  ,1000,0.1)
    k=6
    _ , labels , cc =cv2.kmeans(values , k ,None,criteria,30,cv2.KMEANS_RANDOM_CENTERS)
    cc=np.uint8(cc)
    segmented_image= cc[labels.flatten()]
    segmented_image=segmented_image.reshape(image.shape)
    th3 = cv2.adaptiveThreshold(segmented_image,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
    kernel = np.ones((5,5),np.uint8)
    im4=cv2.erode(th3,kernel,iterations=4)
    plane_mask=points_data['z']
    cv2_img=plane_mask.astype('uint8')
    img=im4
    _,contours, hierarchy = cv2.findContours(im4.astype('uint8'),cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    i=0
    cents=[]
    points=[]
    images=[]
    for i, contour in enumerate(contours):

        area = cv2.contourArea(contour)

        if area > lower and area < higher :
            M = cv2.moments(contour)
            # calculate x,y coordinate of center
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])


            boundRect = cv2.boundingRect(contour)
            #just for drawing rect, dont waste too much time on this
            image_aux= iimmg[boundRect[1]:boundRect[1]+max(boundRect[3],boundRect[2]),boundRect[0]:boundRect[0]+max(boundRect[3],boundRect[2])]
            images.append(image_aux)
            img=cv2.rectangle(img,(boundRect[0], boundRect[1]),(boundRect[0]+boundRect[2], boundRect[1]+boundRect[3]), (0,0,0), 2)
            #img=cv2.rectangle(img,(boundRect[0], boundRect[1]),(boundRect[0]+max(boundRect[2],boundRect[3]), boundRect[1]+max(boundRect[2],boundRect[3])), (0,0,0), 2)
            # calculate moments for each contour
            if (cY > reg_ly and cY < reg_hy and  cX > reg_lx and cX < reg_hx   ):

                cv2.circle(img, (cX, cY), 5, (255, 255, 255), -1)
                cv2.putText(img, "centroid_"+str(i)+"_"+str(cX)+','+str(cY)    ,    (cX - 25, cY - 25)   ,cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 0), 2)
                print ('cX,cY',cX,cY)
                xyz=[]


                for jy in range (boundRect[0], boundRect[0]+boundRect[2]):
                    for ix in range(boundRect[1], boundRect[1]+boundRect[3]):
                        aux=(np.asarray((points_data['x'][ix,jy],points_data['y'][ix,jy],points_data['z'][ix,jy])))
                        if np.isnan(aux[0]) or np.isnan(aux[1]) or np.isnan(aux[2]):
                            'reject point'
                        else:
                            xyz.append(aux)

                xyz=np.asarray(xyz)
                cent=xyz.mean(axis=0)
                cents.append(cent)
                print (cent)
                points.append(xyz)
            else:
                print ('cent out of region... rejected')
                images.pop()
    sub_plt=0
    if plt_images:
        for image in images:

            sub_plt+=1
            ax = plt.subplot(5, 5, sub_plt )

            plt.imshow(image)
            plt.axis("off")

    cents=np.asarray(cents)
    images.append(img)
    return(cents,np.asarray(points), images)


def gaze_point(x,y,z):

    ###Moves head to make center point of rgbd image th coordinates w.r.t.map
    ### To do: (Start from current pose  instead of always going to neutral first )
    
    
    
    head_pose = head.get_current_joint_values()
    head_pose[0]=0.0
    head_pose[1]=0.0
    head.set_joint_value_target(head_pose)
    head.go()
    
    trans , rot = listener.lookupTransform('/map', '/head_rgbd_sensor_gazebo_frame', rospy.Time(0)) #
    
  #  arm_pose=arm.get_current_joint_values()
  #  arm_pose[0]=.1
  #  arm_pose[1]= -0.3
  #  arm.set_joint_value_target(arm_pose)
  #  arm.go()
    
    e =tf.transformations.euler_from_quaternion(rot)
    print('i am at',trans,np.rad2deg(e)[2])
    print('gaze goal',x,y,z)
    #tf.transformations.euler_from_quaternion(rot)


    x_rob,y_rob,z_rob,th_rob= trans[0], trans[1] ,trans[2] ,  e[2]


    D_x=x_rob-x
    D_y=y_rob-y
    D_z=z_rob-z

    D_th= np.arctan2(D_y,D_x)
    print('relative to robot',(D_x,D_y,np.rad2deg(D_th)))

    pan_correct= (- th_rob + D_th + np.pi) % (2*np.pi)

    if(pan_correct > np.pi):
        pan_correct=-2*np.pi+pan_correct
    if(pan_correct < -np.pi):
        pan_correct=2*np.pi+pan_correct

    if ((pan_correct) > .5 * np.pi):
        print ('Exorcist alert')
        pan_correct=.5*np.pi
    head_pose[0]=pan_correct
    tilt_correct=np.arctan2(D_z,np.linalg.norm((D_x,D_y)))

    head_pose [1]=-tilt_correct
    
    
    
    head.set_joint_value_target(head_pose)
    succ=head.go()
    return succ


def move_base(goal_x,goal_y,goal_yaw,time_out=10):

    #using nav client and toyota navigation go to x,y,yaw
    #To Do: PUMAS NAVIGATION
    navclient = actionlib.SimpleActionClient('/move_base/move',MoveBaseAction)
    pose = PoseStamped()
    pose.header.stamp = rospy.Time(0)
    pose.header.frame_id = "map"
    pose.pose.position = Point(goal_x, goal_y, 0)
    quat = tf.transformations.quaternion_from_euler(0, 0, goal_yaw)
    pose.pose.orientation = Quaternion(*quat)


    # create a MOVE BASE GOAL
    goal = MoveBaseGoal()
    goal.target_pose = pose

    # send message to the action server
    navclient.send_goal(goal)

    # wait for the action server to complete the order
    navclient.wait_for_result(timeout=rospy.Duration(time_out))

    # print result of navigation
    action_state = navclient.get_state()
    return navclient.get_state()


def static_tf_publish(cents,index):

    ## Publish tfs of the centroids obtained w.r.t. head sensor frame and references them to map (static)
    t = listener.getLatestCommonTime('/map', '/base_link')
    trans , rot = listener.lookupTransform('/map', '/base_link', t)
    
    #closest_centroid_index=  np.argmin(np.linalg.norm(trans-cents, axis=1))##CLOSEST CENTROID
    closest_centroid_index=0
    min_D_to_base=10
    for  i ,cent  in enumerate(cents):
        x,y,z=cent
        now= rospy.Time.now()
        xyz_map=[]
        cent_quat=[]
        if np.isnan(x) or np.isnan(y) or np.isnan(z):
            print('nan')
        else:
            broadcaster.sendTransform((x,y,z),rot, now, 'Object'+str(index),"head_rgbd_sensor_link")
            rospy.sleep(.2)
            try:
                listener.waitForTransform("/map", 'Object'+str(index), now, rospy.Duration(4.0))
                t = listener.getLatestCommonTime('/map', 'Object'+str(index))
                xyz_map,cent_quat= listener.lookupTransform('/map', 'Object'+str(index),t)
                D_to_base=np.linalg.norm(np.asarray(trans)[:2]-np.asarray(xyz_map)[:2])
                if D_to_base <= min_D_to_base:
                    min_D_to_base=D_to_base
                    closest_centroid_index=i
                    closest_centroid_height= xyz_map[2]
            except (tf.LookupException, tf.ConnectivityException, tf.ExtrapolationException):
                continue

            print ('D Base to obj - ',i, np.linalg.norm(np.asarray(trans)[:2]-np.asarray(xyz_map)[:2]))
    
    i = closest_centroid_index
    ##el problema
    xyz_map,cent_quat= listener.lookupTransform('/map', 'Object'+str(index),rospy.Time(0))
    #print  ('height closest centroid map',xyz_map[2])
    map_euler=tf.transformations.euler_from_quaternion(cent_quat)
    rospy.sleep(.2)
    static_transformStamped = TransformStamped()
            
       
    ##FIXING TF TO MAP ( ODOM REALLY)    
    #tf_broadcaster1.sendTransform( (xyz[0],xyz[1],xyz[2]),tf.transformations.quaternion_from_euler(0, 0, 0), rospy.Time(0), "obj"+str(ind), "head_rgbd_sensor_link")
    static_transformStamped.header.stamp = rospy.Time.now()
    static_transformStamped.header.frame_id = "map"
    
    static_transformStamped.transform.translation.x = float(xyz_map[0])
    static_transformStamped.transform.translation.y = float(xyz_map[1])
    static_transformStamped.transform.translation.z = float(xyz_map[2])
    #quat = tf.transformations.quaternion_from_euler(-euler[0],0,1.5)
    static_transformStamped.transform.rotation.x = 0#-quat[0]#trans.transform.rotation.x
    static_transformStamped.transform.rotation.y = 0#-quat[1]#trans.transform.rotation.y
    static_transformStamped.transform.rotation.z = 0#-quat[2]#trans.transform.rotation.z
    static_transformStamped.transform.rotation.w = 1#-quat[3]#trans.transform.rotation.w
    if xyz_map[1]<-0.6:
        if xyz_map[2] > .7 and xyz_map[2] < .85:
            static_transformStamped.child_frame_id = "Object_0_Table_real_lab"
            tf_static_broadcaster.sendTransform(static_transformStamped)
    else:
        if xyz_map[2] > .45 and xyz_map[2] < .93:   #table 1 
            static_transformStamped.child_frame_id = "Object_"+str(index)+"_Table_1"
            tf_static_broadcaster.sendTransform(static_transformStamped)
        
        if xyz_map[2] > .95 and xyz_map[2] < 1.25:   #table 2 
            static_transformStamped.child_frame_id = "Object_"+str(index)+"_Table_2"
            tf_static_broadcaster.sendTransform(static_transformStamped)
        
        if xyz_map[2] > 1.30 and xyz_map[2] < 1.6:   #table 3 
            static_transformStamped.child_frame_id = "Object_"+str(index)+"_Table_3"
            tf_static_broadcaster.sendTransform(static_transformStamped)
        
    if  xyz_map[2] < .25:   #Floor
        if xyz_map[1] < 1.56:
            static_transformStamped.child_frame_id = "Object_"+str(index)+"_Floor"
            tf_static_broadcaster.sendTransform(static_transformStamped)
           
        if  xyz_map[1] >= 1.56:
            static_transformStamped.child_frame_id = "Object_"+str(index)+"_Floor_"+"risky"
            tf_static_broadcaster.sendTransform(static_transformStamped)
                    
        
    return closest_centroid_height, closest_centroid_index

def move_d_to(target_distance=0.5,target_link='Floor_Object0'):
    ###Face towards Targetlink and get target distance close

    try:
        obj_tar,_ =  listener.lookupTransform('map',target_link,rospy.Time(0))
    except(tf.LookupException):
        print ('no  tf found')
        return False
    
    robot, _ =  listener.lookupTransform('map','base_link',rospy.Time(0))
    pose, quat =  listener.lookupTransform('base_link',target_link,rospy.Time(0))
    euler=euler_from_quaternion(quat)

    D=np.asarray(obj_tar)-np.asarray(robot)
    d=D/np.linalg.norm(D)
    if target_distance==-1:
        new_pose=np.asarray(robot)
    else:
        new_pose=(np.asarray(obj_tar)-target_distance*d)
    
    broadcaster.sendTransform(new_pose,(0,0,0,1), rospy.Time.now(), 'D_from_object','map')
    #for_grasp=new_pose
    #for_grasp[0]+=.15* np.sin(  np.arctan2(pose[1],pose[0])-euler[2])
    #for_grasp[1]+=.15* np.cos( np.arctan2(pose[1],pose[0])-euler[2])
    
    #broadcaster.sendTransform(for_grasp,(0,0,0,1), rospy.Time.now(), 'D_from_object_grasp_floor','map')
    wb_v= whole_body.get_current_joint_values()

    arm.set_named_target('go')
    arm.go()

    print('EULER, WBV', euler_from_quaternion(quat), wb_v[2])
    #succ=move_base( for_grasp[0],for_grasp[1],         np.arctan2(pose[1],pose[0])-euler[2])
    succ=move_base( new_pose[0],new_pose[1],         np.arctan2(pose[1],pose[0])  -euler[2]  -0.07   )
    return succ   
########## Functions for takeshi states ##########
class Proto_state(smach.State):###example of a state definition.
    def __init__(self):
        smach.State.__init__(self,outcomes=['succ','failed','tries'])
        self.tries=0
    def execute(self,userdata):
        rospy.loginfo('State : PROTO_STATE')

        if self.tries==3:
            self.tries=0 
            return'tries'
        if succ:
            return 'succ'
        else:
            return 'failed'
        global trans_hand
        self.tries+=1
        if self.tries==3:
            self.tries=0 
            return'tries'
   
        

    ##### Define state INITIAL #####
#Estado inicial de takeshi, neutral
class Initial(smach.State):
    def __init__(self):
        smach.State.__init__(self,outcomes=['succ','failed','tries'],input_keys=['global_counter'])
        self.tries=0

        
    def execute(self,userdata):

        
        rospy.loginfo('STATE : robot neutral pose')
        print('Try',self.tries,'of 5 attepmpts') 
        self.tries+=1
        if self.tries==3:
            return 'tries'
        clear_octo_client()
        scene.remove_world_object()
        #Takeshi neutral
        arm.set_named_target('go')
        arm.go()
        head.set_named_target('neutral')
        succ = head.go()             
        if succ:
            return 'succ'
        else:
            return 'failed'
class Turn_around(smach.State):
    def __init__(self):
        smach.State.__init__(self,outcomes=['succ','failed','tries'])
        self.tries=0
    def execute(self,userdata):
        rospy.loginfo('State : Turn around ')
        print('Try',self.tries,'of 5 attepmpts') 
        self.tries+=1
        if self.tries==3:
            return 'tries'
        clear_octo_client()
        scene.remove_world_object()
        #Takeshi turns around and then neutral
        move_base(0.0,0.0,3.14159)
        head.set_named_target('neutral')
        succ = head.go()             
        if succ:
            return 'succ'
        else:
            return 'failed'
class Self_locate(smach.State):
    def __init__(self):
        smach.State.__init__(self,outcomes=['succ','failed','tries'])
        self.tries=0
    def execute(self,userdata):
        rospy.loginfo('State : Self_locate ')
        print('Try',self.tries,'of 5 attepmpts') 
        self.tries+=1
        if self.tries==3:
            return 'tries'
        clear_octo_client()
        scene.remove_world_object()
        #static_tf_publish_furniture(-0.95,-0.5,0.0)
        gaze_point(-0.3,-0.7,0.80)
        cents,xyz, images=seg_square_imgs(lower=8000,higher=10000,plt_images=True)
        cch,cci=static_tf_publish(cents,0)
        if len(images)!=0:
            succ=True
        else:
            succ=False
        head.set_named_target('neutral')
        head.go()
        if succ:
            return 'succ'
        else:
            return 'failed'
        
class Goto_shelf(smach.State):
    def __init__(self):
        smach.State.__init__(self,outcomes=['succ','failed','tries'])
        self.tries=0
    def execute(self,userdata):
        rospy.loginfo('State : GOTO_SHELF')
        print('Try',self.tries,'of 5 attepmpts') 
        self.tries+=1
        if self.tries==3: 
            return'tries'
        clear_octo_client()
        scene.remove_world_object()
        #Takeshi gets close to the shelf
        move_base(-0.4,0.0,3.14159)
        head.set_named_target('neutral')
        succ=head.go()
        if succ:
            return 'succ'
        else:
            return 'failed'
class Scan_shelf(smach.State):
    def __init__(self):
        smach.State.__init__(self,outcomes=['succ','failed','tries'])
        self.tries=0
    def execute(self,userdata):
        rospy.loginfo('State : SCAN_SHELF')
        print('Try',self.tries,'of 5 attepmpts') 
        self.tries+=1
        if self.tries==3: 
            return'tries'
        clear_octo_client()
        scene.remove_world_object()
        #Takeshi scans the shelf
        #gaze_point(-2.0,-0.0,1.5)
        goal=[0.15, None,-1.57079,None,None, None]
        move_to_joint_value(goal)
        try:
            succ=False
            rospy.sleep(0.65)
            cents,xyz,images=seg_square_imgs(reg_ly=250,plt_images=True)
            #print(cents)
            rospy.sleep(0.65)
            if len(images) != 0:
        #print(cents[1][0])
                static_tf_publish(cents,1)
                succ=True
        except:
            print("failed")
            suc=False
        if succ:
            return 'succ'
        else:
            return 'failed'
class Pre_grasp_shelf(smach.State):###get a convenient pre grasp pose
    def __init__(self):
        smach.State.__init__(self,outcomes=['succ','failed','tries'])
        self.tries=0
    def execute(self,userdata):
        rospy.loginfo('State : PRE_GRASP_SHELF')
        print ("self.tries",self.tries)
        #target_tf= 'Object_0'
        target_tf='Object_1_Table_2'
        move_d_to(0.3,target_tf)
        head.set_named_target('neutral')
        head.go()
        arm.set_named_target('neutral')
        arm.go()
        #move_to_pose_value(posZ=cch)
        #open_gripper()      
    
        pose,quat= listener.lookupTransform('/base_link',target_tf,rospy.Time(0))
        pose[0]+=-0.15
        #pose[1]+= 0.05
        broadcaster.sendTransform(pose, quat,rospy.Time.now(),'Pre_grasp','base_link')
        rospy.sleep(.1)    
     
        try:
            xyz_map, quat =  listener.lookupTransform('map','Pre_grasp',rospy.Time(0))
            print(xyz_map)
        except(tf.LookupException):
            print ('no pre grasp table2 tf')
            self.tries+=1
        #return 'failed'
        
        clear_octo_client()
        move_to_pose_value(posZ=xyz_map[2])
        open_gripper()
        pose=arm.get_current_pose().pose.position
        move_to_pose_value(CG=0,posZ=pose.z+0.05,posX=pose.x-0.25)   
 
        try:
            xyz_map, quat =  	listener.lookupTransform('map','Pre_grasp',rospy.Time(0))
            print(xyz_map)
            succ = True
        except(tf.LookupException):
            print ('no pre grasp table2 tf')
            self.tries+=1
            succ=False
        
        
        clear_octo_client()
        move_to_pose_value(posZ=xyz_map[2])
        open_gripper()
        pose=arm.get_current_pose().pose.position
        move_to_pose_value(CG=0,posZ=pose.z+0.05,posX=pose.x-0.25)
    
        if succ:
            return 'succ'
        else:
            return 'failed'
       
##################################################Pre_grasp_floor()      
class Grasp_shelf(smach.State):###example of a state definition.
    def __init__(self):
        smach.State.__init__(self,outcomes=['succ','failed','tries'])
        self.tries=0
    def execute(self,userdata):
        rospy.loginfo('State : GRASP_SHELF')
        target_tf='Object_1_Table_2'
        pose, quat =  listener.lookupTransform('hand_palm_link',target_tf,rospy.Time(0))
        print(pose)
        while abs(pose[2]) > 0.15:
            pose, quat =  listener.lookupTransform('hand_palm_link',target_tf,rospy.Time(0))
            #robot_pose = whole_body.get_current_pose().pose.position
            #delta =pose[0]-robot_pose.x
            #print(delta)
            if pose [1] >- 0.2 and pose[1]<0.02:
                print ('getting close')
                move_abs(0.05,0,0,0.1)
            if pose[1] >= 0.02:
                print ('drift correct   -')
                move_abs(0.0,-0.05,-10, 0.10) 
            if pose[1] <= -0.02:
                print ('drift correct   +')
                move_abs(0.0, 0.05,10, 0.10) #GRADOS! WTF , 
        rospy.sleep(0.1)
        clear_octo_client()
        #move_to_pose_value(posZ=xyz_map[2])
        #open_gripper()
        #move_to_pose_value(CG=1,posY=xyz_map[1])
        close_gripper()
        ajv = arm.get_current_joint_values()
        print(ajv)
        ajv[4]=-1
        move_to_joint_value(ajv)
        goal=whole_body.get_current_pose()

        move_to_pose_value(posZ=goal.pose.position.z+0.10)
        succ=True

    
        if succ:
            return 'succ'
        else:
            return 'failed'

class Safe_pos(smach.State):
    def __init__(self):
        smach.State.__init__(self,outcomes=['succ','failed','tries'],input_keys=['global_counter'])
        self.tries=0

        
    def execute(self,userdata):

        
        rospy.loginfo('STATE : robot neutral pose')
        print('Try',self.tries,'of 5 attepmpts') 
        self.tries+=1
        if self.tries==3:
            return 'tries'
        clear_octo_client()
        scene.remove_world_object()
        #Takeshi neutral
        arm.set_named_target('go')
        arm.go()
        head.set_named_target('neutral')
        succ = head.go()             
        if succ:
            return 'succ'
        else:
            return 'failed'
class Goto_play(smach.State):
    def __init__(self):
        smach.State.__init__(self,outcomes=['succ','failed','tries'],input_keys=['global_counter'])
        self.tries=0

        
    def execute(self,userdata):

        
        rospy.loginfo('STATE : robot neutral pose')
        print('Try',self.tries,'of 5 attepmpts') 
        self.tries+=1
        if self.tries==3:
            return 'tries'
        clear_octo_client()
        scene.remove_world_object()
        #Takeshi neutral
        arm.set_named_target('go')
        arm.go()
        head.set_named_target('neutral')
        succ = head.go()             
        if succ:
            return 'succ'
        else:
            return 'failed'

class Leave_cas(smach.State):
    def __init__(self):
        smach.State.__init__(self,outcomes=['succ','failed','tries'],input_keys=['global_counter'])
        self.tries=0

        
    def execute(self,userdata):

        
        rospy.loginfo('STATE : robot neutral pose')
        print('Try',self.tries,'of 5 attepmpts') 
        self.tries+=1
        if self.tries==3:
            return 'tries'
        clear_octo_client()
        scene.remove_world_object()
        #Takeshi neutral
        arm.set_named_target('go')
        arm.go()
        head.set_named_target('neutral')
        succ = head.go()             
        if succ:
            return 'succ'
        else:
            return 'failed'
        
#Initialize global variables and node
def init(node_name):
    global listener, broadcaster, tfBuffer, tf_static_broadcaster, scene, rgbd, gripper,head,whole_body,arm,goal,navclient,clear_octo_client,service_client,base_vel_pub
    moveit_commander.roscpp_initialize(sys.argv)
    rospy.init_node(node_name)
    
    head = moveit_commander.MoveGroupCommander('head')
    gripper = moveit_commander.MoveGroupCommander('gripper')
#     gripper =  moveit_commander.MoveGroupCommander('gripper')
    whole_body=moveit_commander.MoveGroupCommander('whole_body_light')
    arm =  moveit_commander.MoveGroupCommander('arm')
    
    listener = tf.TransformListener()
    broadcaster = tf.TransformBroadcaster()
    tfBuffer = tf2_ros.Buffer()
    tf_static_broadcaster = tf2_ros.StaticTransformBroadcaster()
    whole_body.set_workspace([-6.0, -6.0, 6.0, 6.0]) 
    
    scene = moveit_commander.PlanningSceneInterface()
    robot = moveit_commander.RobotCommander()
    rgbd = RGBD()
    goal = MoveBaseGoal()
    
    navclient = actionlib.SimpleActionClient('/move_base/move', MoveBaseAction)
    clear_octo_client = rospy.ServiceProxy('/clear_octomap', Empty)
    service_client = rospy.ServiceProxy('/segment_2_tf', Trigger)
    #service_client.wait_for_service(timeout=1.0)
    base_vel_pub = rospy.Publisher('/hsrb/command_velocity', Twist, queue_size=10)
   


#Entry point    
if __name__== '__main__':
    print("Takeshi STATE MACHINE...")
    init("takeshi_smach")
    sm = smach.StateMachine(outcomes = ['END'])     #State machine, final state "END"
    sm.userdata.sm_counter = 0

    with sm:
        #State machine for grasping on Floor
        smach.StateMachine.add("INITIAL",Initial(),transitions = {'failed':'INITIAL',      'succ':'TURN_AROUND',    'tries':'END'}) 
        smach.StateMachine.add("TURN_AROUND",Turn_around(),transitions = {'failed':'END',      'succ':'GOTO_SHELF',    'tries':'TURN_AROUND'}) 
        smach.StateMachine.add("GOTO_SHELF",Goto_shelf(),transitions = {'failed':'END',      'succ':'SCAN_SHELF',    'tries':'GOTO_SHELF'}) 
        smach.StateMachine.add("SCAN_SHELF",Scan_shelf(),transitions = {'failed':'SCAN_SHELF',      'succ':'PRE_GRASP_SHELF',    'tries':'SCAN_SHELF'}) 
        smach.StateMachine.add("PRE_GRASP_SHELF",Pre_grasp_shelf(),transitions = {'failed':'END',      'succ':'GRASP_SHELF',    'tries':'PRE_GRASP_SHELF'}) 

        smach.StateMachine.add("GRASP_SHELF",Grasp_shelf(),transitions = {'failed':'GRASP_SHELF',      'succ':'SAFE_POS',    'tries':'GRASP_SHELF'}) 
        smach.StateMachine.add("SAFE_POS",Safe_pos(),transitions = {'failed':'GRASP_SHELF',      'succ':'GOTO_PLAY',    'tries':'GRASP_SHELF'}) 
        smach.StateMachine.add("GOTO_PLAY",Goto_play(),transitions = {'failed':'GRASP_SHELF',      'succ':'LEAVE_CAS',    'tries':'GRASP_SHELF'}) 
        smach.StateMachine.add("LEAVE_CAS",Leave_cas(),transitions = {'failed':'GRASP_SHELF',      'succ':'FINAL',    'tries':'GRASP_SHELF'}) 
        smach.StateMachine.add("FINAL",Initial(),transitions = {'failed':'INITIAL',      'succ':'END',    'tries':'END'}) 

      

    outcome = sm.execute()



Takeshi STATE MACHINE...


[ WARN] [1660507597.982264191, 100.392000000]: Link hand_l_finger_vacuum_frame has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying collision geometry.
[ WARN] [1660507597.986952262, 100.392000000]: Link head_l_stereo_camera_link has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying collision geometry.
[ WARN] [1660507597.987046446, 100.392000000]: Link head_r_stereo_camera_link has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying collision geometry.
[ WARN] [1660507597.988274232, 100.392000000]: Group state 'neutral' doesn't specify all group joints in group 'arm'. wrist_ft_sensor_frame_joint is missing.
[ WARN] [1660507597.988452132, 100.392000000]: Group state 'go' doesn't specify all group joints in group 'arm'. wrist_ft_sensor_frame_joint is missing.
[ WA

RuntimeError: Unable to connect to move_group action server 'execute_trajectory' within allotted time (5s)

[ WARN] [1660507608.906806700, 101.805000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 101.904000 according to authority unknown_publisher
[ WARN] [1660507612.566243655, 102.552000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 102.600000 according to authority unknown_publisher
[ WARN] [1660507616.320062583, 103.293000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 103.320000 according to authority unknown_publisher
[ WARN] [1660507621.437424608, 104.139000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 104.247000 according to authority unknown_publisher
[ WARN] [1660507624.557318844, 104.886000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 104.886000 according to authority unknown_publisher
[ WARN] [1660507646.028371441, 108.285000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for fra

[ WARN] [1660507736.767943496, 122.787000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 122.835000 according to authority unknown_publisher
[ WARN] [1660507754.266943806, 125.340000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 125.385000 according to authority unknown_publisher
[ WARN] [1660507762.340445813, 126.771000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 126.825000 according to authority unknown_publisher
[ WARN] [1660507767.136698336, 127.527000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 127.575000 according to authority unknown_publisher
[ WARN] [1660507778.370659782, 129.372000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 129.423000 according to authority unknown_publisher
[ WARN] [1660507808.119228002, 134.727000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for fra

[ WARN] [1660508230.588324617, 209.865000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 209.931000 according to authority unknown_publisher
[ WARN] [1660508247.373378719, 212.445000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 212.490000 according to authority unknown_publisher
[ WARN] [1660508253.748566999, 213.780000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 213.825000 according to authority unknown_publisher
[ WARN] [1660508254.694164257, 213.981000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 214.032000 according to authority unknown_publisher
[ WARN] [1660508269.970589336, 216.684000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 216.720000 according to authority unknown_publisher
[ WARN] [1660508277.548382812, 217.884000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for fra

[ WARN] [1660508657.541645351, 281.100000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 281.130000 according to authority unknown_publisher
[ WARN] [1660508669.377130570, 283.110000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 283.149000 according to authority unknown_publisher
[ WARN] [1660508669.920912748, 283.200000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 283.227000 according to authority unknown_publisher
[ WARN] [1660508675.097835418, 284.046000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 284.073000 according to authority unknown_publisher
[ WARN] [1660508676.801226468, 284.373000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 284.430000 according to authority unknown_publisher
[ WARN] [1660508682.578343773, 285.303000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for fra

In [ ]:
 pose, quat =  listener.lookupTransform('hand_palm_link',target_tf,rospy.Time(0))
print(pose,quat)

In [ ]:
#go to init
#close_gripper()
head.set_named_target('neutral')
head.go()
arm.set_named_target('go')
arm.go()
move_base(0.0,0.0,0.0)

In [ ]:
#turn around
clear_octo_client()
scene.remove_world_object()
#Takeshi turns around and then neutral
move_base(0.0,0.0,3.14159)
head.set_named_target('neutral')
succ = head.go()  

In [ ]:
#go to shelf
clear_octo_client()
scene.remove_world_object()
#Takeshi gets close to the shelf
move_base(-0.4,0.0,3.14159)
head.set_named_target('neutral')
succ=head.go()

In [ ]:
#scan shelf
clear_octo_client()
scene.remove_world_object()
#Takeshi scans the shelf
#gaze_point(-2.0,-0.0,1.5)
goal=[0.15, None,-1.57079,None,None, None]
move_to_joint_value(goal)
#while True:
succ=False
try:
    cents,_,images=seg_square_imgs(reg_ly=250,plt_images=True)
#print(cents)
    if len(images) != 0:
#print(cents[1][0])
        static_tf_publish(cents,1)
        succ=True
except():
    print("no se que pasa")


In [ ]:
#pre grasp shelf
target_tf='Object_1_Table_2'
move_d_to(0.3,target_tf)
head.set_named_target('neutral')
head.go()
arm.set_named_target('neutral')
arm.go()
#move_to_pose_value(posZ=cch)
#open_gripper()      

pose,quat= listener.lookupTransform('/base_link',target_tf,rospy.Time(0))
pose[0]+=-0.15
#pose[1]+= 0.05
broadcaster.sendTransform(pose, quat,rospy.Time.now(),'Pre_grasp','base_link')
rospy.sleep(.1)    
 
try:
    xyz_map, quat =  listener.lookupTransform('map','Pre_grasp',rospy.Time(0))
    print(xyz_map)
except(tf.LookupException):
    print ('no pre grasp table2 tf')
    #self.tries+=1
    #return 'failed'
        
clear_octo_client()
move_to_pose_value(posZ=xyz_map[2])
open_gripper()
pose=arm.get_current_pose().pose.position
move_to_pose_value(CG=0,posZ=pose.z+0.05,posX=pose.x-0.25)
#move_to_pose_value(CG=1,posY=xyz_map[1])

In [ ]:
 pose, quat =  listener.lookupTransform('hand_palm_link',target_tf,rospy.Time(0))
print(pose)
pose=whole_body.get_current_pose()
print(pose)
pose=arm.get_current_pose()
print(pose)
#move_to_pose_value(CG=1,posY=xyz_map[1])


In [ ]:
target_tf='Object_1_Table_2'
pose, quat =  listener.lookupTransform('hand_palm_link',target_tf,rospy.Time(0))
print(pose)
while abs(pose[2]) > 0.15:
    pose, quat =  listener.lookupTransform('hand_palm_link',target_tf,rospy.Time(0))
    #robot_pose = whole_body.get_current_pose().pose.position
    #delta =pose[0]-robot_pose.x
    #print(delta)
    if pose [1] >- 0.2 and pose[1]<0.02:
        print ('getting close')
        move_abs(0.05,0,0,0.1)
    if pose[1] >= 0.02:
        print ('drift correct   -')
        move_abs(0.0,-0.05,-10, 0.10) 
    if pose[1] <= -0.02:
        print ('drift correct   +')
        move_abs(0.0, 0.05,10, 0.10) #GRADOS! WTF , 
rospy.sleep(0.1)

In [ ]:
move_base(0,0,0)
#close_gripper()

In [ ]:
#grasp
#target_tf='Object_1_Table_2'
#pose,quat= listener.lookupTransform('/base_link',target_tf,rospy.Time(0))
#try:
#    xyz_map, quat =  listener.lookupTransform('map',target_tf,rospy.Time(0))
#    print(xyz_map)
#except(tf.LookupException):
#    print ('no grasp table2 tf')
    #self.tries+=1
    #return 'failed'
        
clear_octo_client()
#move_to_pose_value(posZ=xyz_map[2])
#open_gripper()
#move_to_pose_value(CG=1,posY=xyz_map[1])
close_gripper()
ajv = arm.get_current_joint_values()
print(ajv)
ajv[4]=-1
move_to_joint_value(ajv)
goal=whole_body.get_current_pose()

move_to_pose_value(posZ=goal.pose.position.z+0.10)



In [ ]:
move_base(-0.5,0.0,3.1415)

In [ ]:
#move_to_pose_value(CG=0,posX=xyz_map[0]-0.30)
arm.set_named_target('go')
arm.go()
move_base(-0.5,0.0,-1.5707)
#open_gripper()

In [ ]:
move_base(-0.5,-0.3,-1.5707)
head.set_named_target('neutral')
head.go()

In [ ]:
place_casete=[0.20,None,0.0,0.0,1.5707,0.0]
move_to_joint_value(place_casete)
place_casete[3]=-1.5707
move_to_joint_value(place_casete)

In [ ]:
place=arm.get_current_pose()
place
move_to_pose_value(posZ=place.pose.position.z+0.05)
move_to_pose_value(posY=place.pose.position.y-0.15)
#


In [ ]:
place=arm.get_current_pose()
move_to_pose_value(posZ=place.pose.position.z-0.05)

In [ ]:
open_gripper()

In [ ]:
#open_gripper()
#retire=whole_body.get_current_pose()
#move_to_pose_value(CG=0,posY=retire.pose.position.y+0.05)
move_base(-0.5,-0.0,-1.5707)
close_gripper()

In [ ]:
pID=arm.set_planner_id("RRTstarkConfigDefault")
#pID=arm.get_planner_id()
#print("the planner ID is"+pID)